# Loading Polygon Data

Creating a connection the gridMET planetary dataset

In [1]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
asset

<Asset href=abfs://gridmet/gridmet.zarr>

Using xarray to read it:

In [2]:
import xarray as xr

ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)
ds

<xarray.Dataset>
Dimensions:                                    (time: 15341, lat: 585,
                                                lon: 1386, crs: 1)
Coordinates:
  * crs                                        (crs) uint16 3
  * lat                                        (lat) float64 49.4 ... 25.07
  * lon                                        (lon) float64 -124.8 ... -67.06
  * time                                       (time) datetime64[ns] 1979-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    potential_evapotranspiration               (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    precipitation_amount                       (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    relative_humidity                          (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    specific_humidity                          (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_from_direction                        (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_speed                                 (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

Creating the Chicago polygon and time period:

In [ ]:
polygon_dict = {
    'california': {
        'lat_min': 32.5,
        'lat_max': 42.0,
        'lon_min': -124.5,
        'lon_max': -114.1
    },
    'chicago': {
        'lat_min': 41.5781,
        'lat_max': 42.1781,
        'lon_min': -87.9298,
        'lon_max': -87.3298
    },
    'south_carolina': {
        'lat_min': 32.0,
        'lat_max': 35.2,
        'lon_min': -83.4,
        'lon_max': -78.5
    }
}


# Select which polygon to use
sel_polygon = 'south_carolina' 

# Extract bounding values from the dictionary
lat_min = polygon_dict[sel_polygon]['lat_min']
lat_max = polygon_dict[sel_polygon]['lat_max']
lon_min = polygon_dict[sel_polygon]['lon_min']
lon_max = polygon_dict[sel_polygon]['lon_max']


ds_polygon = ds.sel(
    time=slice("2017-01-01", "2020-12-31"),
    lat=slice(lat_max, lat_min),  # Note: latitudes are descending
    lon=slice(lon_min, lon_max)
)

# Preview the selection
ds_polygon


<xarray.Dataset>
Dimensions:                                    (time: 1461, lat: 228, lon: 249,
                                                crs: 1)
Coordinates:
  * crs                                        (crs) uint16 3
  * lat                                        (lat) float64 41.98 ... 32.53
  * lon                                        (lon) float64 -124.5 ... -114.1
  * time                                       (time) datetime64[ns] 2017-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    potential_evapotranspiration               (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    precipitation_amount                       (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    relative_humidity                          (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    specific_humidity                          (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    wind_from_direction                        (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
    wind_speed                                 (time, lat, lon) float32 dask.array<chunksize=(10, 228, 249), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [16]:
# Estimate memory usage of the subset
total_bytes = sum(v.data.nbytes for v in ds_polygon.data_vars.values())
print(f"Estimated memory size for {sel_polygon} polygon_data: {total_bytes / 1e9:.2f} GB")


Estimated memory size for california polygon_data: 3.98 GB


## Weekly resampling each pair of (lat,long)

In [6]:
agg_methods = {
    "air_temperature": "mean",  # Average temperature per week
    "burning_index_g": "max",   # Use max for fire danger indicators
    "dead_fuel_moisture_1000hr": "mean",  # Slow-changing moisture → mean
    "dead_fuel_moisture_100hr": "mean",   # Same as above
    "mean_vapor_pressure_deficit": "mean",  # Continuous variable → mean
    "potential_evapotranspiration": "sum",  # Accumulated over time
    "precipitation_amount": "sum",  # Rainfall is cumulative
    "relative_humidity": "mean",  # Weekly average humidity
    "specific_humidity": "mean",  # Same idea
    "surface_downwelling_shortwave_flux_in_air": "mean",  # Radiative flux → mean
    "wind_from_direction": "mean",  # Can use circular stats, but mean works as approx
    "wind_speed": "mean"  # Weekly average wind speed
}

In [7]:
# Step 1: Extract only relevant variables
ds_subset = ds_polygon[list(agg_methods.keys())]

# Step 2: Create a new Dataset by applying appropriate method
ds_weekly = xr.Dataset({
    var: getattr(ds_subset[var].resample(time="1W"), method)()
    for var, method in agg_methods.items()
})

# Step 3 (optional): Inspect result
print(ds_weekly)

<xarray.Dataset>
Dimensions:                                    (lat: 228, lon: 249, time: 210)
Coordinates:
  * lat                                        (lat) float64 41.98 ... 32.53
  * lon                                        (lon) float64 -124.5 ... -114.1
  * time                                       (time) datetime64[ns] 2017-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(1, 228, 249), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(1, 228, 249), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(1, 228, 249), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(1, 228, 249), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(1, 228, 249), meta=np.ndarray>
    potential

If we were to do this:
```python
df_weekly = ds_weekly.to_dataframe().reset_index()
``` 
gives: One row per week per lat-lon pair.

So if you have 14 latitudes × 14 longitudes = 196 grid points, you’ll get:

`210 weeks × 196 = 41,160 rows`

Each row corresponds to the climate values at a specific grid cell in Chicago for a specific week.

If we aggregate it for the **whole of Chicago:**

In [8]:
ds_weekly_avg = ds_weekly.mean(dim=["lat", "lon"])

## DataFraming it

Let's convert it into an actual DataFrame:

In [9]:
df_weekly_avg = ds_weekly_avg.to_dataframe().reset_index()

c:\Users\lucas\anaconda3\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


In [10]:
df_weekly_avg

,time,air_temperature,burning_index_g,dead_fuel_moisture_1000hr,dead_fuel_moisture_100hr,mean_vapor_pressure_deficit,potential_evapotranspiration,precipitation_amount,relative_humidity,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,wind_speed
0,2017-01-01,269.565735,9.550136,25.258408,20.629345,0.223527,0.756774,1.184878,52.242970,0.003714,104.119781,234.677795,3.699810
1,2017-01-08,270.231934,15.742865,25.535362,20.943819,0.240263,5.514424,55.745010,56.208645,0.004018,81.207329,177.593979,4.480159
2,2017-01-15,273.045837,18.572361,26.072039,19.996796,0.293467,6.469516,31.089275,54.818554,0.004161,97.528381,181.833694,4.235615
3,2017-01-22,271.917938,20.511511,24.633095,18.202560,0.301167,6.598039,50.196251,51.700916,0.003706,93.443977,183.125488,4.466816
4,2017-01-29,268.559509,24.447126,24.493650,17.806400,0.383847,7.597791,3.634114,39.588493,0.002817,123.849205,193.801422,3.109412
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2020-12-06,271.396393,53.943619,11.593079,10.005462,0.800662,16.839767,0.230207,17.395708,0.002152,118.309235,153.983780,2.757477
206,2020-12-13,271.888977,57.500305,11.262872,11.001671,0.642507,15.070644,8.621865,29.340914,0.002795,99.552040,185.076431,3.240357
207,2020-12-20,271.450745,42.664608,13.499342,15.992971,0.477459,11.721825,6.437439,38.162201,0.003211,102.237366,227.501434,3.113607
208,2020-12-27,271.335632,50.172657,13.649246,12.874249,0.589699,14.156526,6.671551,29.450277,0.002672,102.658897,165.184433,3.412922


In [ ]:
csv_path = f"data/{sel_polygon}_weekly_avg.csv"
parquet_path = f"data/{sel_polygon}_weekly_avg.parquet"

df_weekly_avg.to_csv(csv_path, index=False)
df_weekly_avg.to_parquet(parquet_path, index=False)

print(f"✅ Data saved to:\n- {csv_path}\n- {parquet_path}")

✅ Data saved to:
- data/california_weekly_avg.csv
- data/california_weekly_avg.parquet
✅ Data saved as CSV and Parquet!
